# Projeto 1 - Ciência dos Dados

Nome: Eric de Araújo Lima Barbosa

Nome: Lucca de Paiva Barroso Saraiva

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lucca\Desktop\Projeto1_ciencia_dados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'netflix1.2.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  classificação
0  namoral as serie da hbo max pisa samba rebola ...              1
1  rt @xd_xandao: @gaguinho001 porra tlgd kkkkkkk...              1
2  rt @___dudasilvaa: eu e juan vimos todos os fi...              0
3  com a criação destes streaminga doa estúdios d...              1
4  gosto da amazon prime, mais tem tanta série qu...              0

In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  classificação
0  rt @notthatindigo: sim, eu me garanto sem filt...              0
1  leitores morrem após não receber nenhuma propo...              0
2  @netflixbrasil se vc tirar naruto vc me paga n...              1
3  faltou o loirinho pra assistir netflix cmg #askbf              0
4  netflix se as personagens são portugueses não ...              1

___
## Classificador automático de sentimento




**O produto escolhido foi a plataforma de streaming de filmes e séries, Netflix. Para esse projeto, consideramos como relevantes 
quaisquer tweets que mencionassem a marca netflix e títulos atrelados à ela, incluindo: Críticas à plataforma, comparações com 
concorrentes, elogios à plataforma e críticas e elogios direcionados à séries presentes no catálogo pela netflix.**

___
### Montando um Classificador Naive-Bayes



In [6]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;\n]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
#Separando o dataframe em dois considerando a classificação(1->RELEVANTE/0->IRRELEVANTE)
tweet_train = train.Treinamento

tweet_train_rel = train.loc[train['classificação']== 1,:]
tweet_train_rel = tweet_train_rel.Treinamento



tweet_train_irrel = train.loc[train['classificação']== 0,:]
tweet_train_irrel = tweet_train_irrel.Treinamento



In [8]:
#limpando todos os tweets da base de treino e criando uma série que contenha todas as palavras
tweets = []
for texto in tweet_train:
    
    tweet_raw = cleanup(texto.lower()).split()
    
    
    for palavra in tweet_raw:
        #limpa os 'rt's e os links
        if palavra != 'rt' and palavra[:4] != 'http':
            tweets.append(palavra)
        else:
            pass
        



tweets_tot_serie = pd.Series(tweets)






In [9]:
#limpando os tweets relevantes da base de treino e criando uma série que contenha todas as palavras dos tweets relevantes
tweets_rel = []
for texto in tweet_train_rel:
    
    tweet_raw = cleanup(texto.lower()).split()
    
    
    for palavra in tweet_raw:
        #limpa os 'rt's e os links
        if palavra != 'rt' and palavra[:4] != 'http':
            tweets_rel.append(palavra)
        else:
            pass

        
#Total de palavras contando os tweets RELEVANTES(leva em conta palavras repetidas)
print(len(tweets_rel))

2774


In [10]:
#Serie com todas as palavras de tweets relevantes
tweets_rel_serie = pd.Series(tweets_rel)

#contagem do numero de vezes que cada palavra aparece em tweets relevantes
tweets_rel_serie.value_counts()

netflix       140
a              78
de             73
que            62
e              60
             ... 
quase           1
indicações      1
foram           1
realizar        1
👏🏻🙏🏻            1
Length: 1108, dtype: int64

In [11]:
#limpando os tweets irrelevantes da base de treino e criando uma série que contenha todas as palavras dos tweets irrelevantes
tweets_irrel = []
for texto in tweet_train_irrel:
    
    tweet_raw = cleanup(texto.lower()).split()
    
    
    for palavra in tweet_raw:
        #limpa os 'rt's e os links
        if palavra != 'rt' and palavra[:4] != 'http':
            tweets_irrel.append(palavra)
        else:
            pass

        
#Total de palavras contando os tweets IRRELEVANTES(leva em conta palavras repetidas)
print(len(tweets_irrel))

2788


In [12]:
#Serie com todas as palavras de tweets relevantes
tweets_irrel_serie = pd.Series(tweets_irrel)

#contagem do numero de vezes que cada palavra aparece em tweets relevantes
tweets_irrel_serie.value_counts()

netflix        137
e               78
a               75
que             69
de              58
              ... 
cpa              1
tarde            1
netflixamor      1
@electro31p      1
voltar           1
Length: 1166, dtype: int64

In [13]:
#probabilidade de um tweet ser relevante (nº de tweets relevantes na base de treino/total de tweets na base de treino)
P_rel = len(tweet_train_rel)/len(tweet_train)

#probabilidade de um tweet ser irrelevante (nº de tweets irrelevantes na base de treino/total de tweets na base de treino)
P_irrel = len(tweet_train_irrel)/len(tweet_train)




In [14]:
#Definindo uma função que calcula a probabilidade de um dado tweet ser relevante
def P_rel_dado_tweet(tweet):
    separado = tweet.split()
    P_tweet_dado_rel = 1
    for palavra in separado:
        if palavra in tweets_rel_serie.value_counts():
            P_tweet_dado_rel = P_tweet_dado_rel*((tweets_rel_serie.value_counts()[palavra]+1)/(len(tweets_rel)+len(tweets_tot_serie.value_counts())))
        
            
        else:
            P_tweet_dado_rel = P_tweet_dado_rel*((0+1)/(len(tweets_rel)+len(tweets_tot_serie.value_counts())))
        
    resultado = P_tweet_dado_rel * P_rel
            
    
    return resultado

In [15]:
#Definindo uma função que calcula a probabilidade de um dado tweet ser irrelevante
def P_irrel_dado_tweet(tweet):
    separado = tweet.split()
    P_tweet_dado_irrel = 1
    for palavra in separado:
        if palavra in tweets_irrel_serie.value_counts():
            P_tweet_dado_irrel = P_tweet_dado_irrel*((tweets_irrel_serie.value_counts()[palavra]+1)/(len(tweets_irrel)+len(tweets_tot_serie.value_counts())))
        
            
        else:
            P_tweet_dado_irrel = P_tweet_dado_irrel*((0+1)/(len(tweets_irrel)+len(tweets_tot_serie.value_counts())))
            
    resultado = P_tweet_dado_irrel * P_irrel
            
    return resultado



___
### Verificando a performance do Classificador



In [16]:
#preparando a base de testes para o teste de performance(aplicando limpezas)
a_final = []
for tweet in test.Teste:
    a = cleanup(tweet)
    b = a.split()
    a_limpo = []
    
    
    
    
    for palavra in b:
            #limpa os 'rt's e os links
        if palavra != 'rt' and palavra[:4] != 'http':
            a_limpo.append(palavra)
        else:
            pass
    a_limpo_tot = ' '.join(a_limpo)

    a_final.append(a_limpo_tot)
    
    
#criando um dataframe com os tweets da base de testes já limpos
test['tweets'] = a_final

test_final = test.loc[:, ['tweets', 'classificação']]

test_final.head(10)




tweets  classificação
0  @notthatindigo sim eu me garanto sem filtro tw...              0
1  leitores morrem após não receber nenhuma propo...              0
2  @netflixbrasil se vc tirar naruto vc me paga n...              1
3   faltou o loirinho pra assistir netflix cmg askbf              0
4  netflix se as personagens são portugueses não ...              1
5  primeira imagem conceitual de troll filme da n...              0
6  @omelete idris elba e caleb mclaughlin têm rel...              1
7  @hbomaxbr_portal gosto de acreditar que vem no...              1
8  @ruifilipe993 @netflix essa série é incrível e...              1
9  @kozumearts vê blue exorcist tem na netflix e ...              1

In [17]:
#importando a biblioteca random do python
import random 

#Definindo uma função que classifica um dado tweet comparando sua probabilidade de ser relevante e irrelevante
def classifica(tweet):
    p1 = P_rel_dado_tweet(tweet)
    p0=P_irrel_dado_tweet(tweet)
    if p1>p0:
        return '1' #se for mais provável do tweet ser relevante retorna a classificação 1
    elif p0>p1:
        return '0' #se for mais provável do tweet ser irrelevante retorna a classificação 0
    else:
        return random.randint(0,1) #se as probabilidades forem exatamente iguais, escolhe-se aleatoriamente uma classificação 

#roda o teste e cria uma nova coluna no dataframe de teste limpo que indica a classificação dada pelo classificador
class_teste = []    
for tweet in test_final['tweets']:
    resultado = classifica(tweet)
    class_teste.append(resultado)
    

test_final['teste'] = class_teste
test_final   
    
        
    

        
        

tweets  classificação teste
0    @notthatindigo sim eu me garanto sem filtro tw...              0     0
1    leitores morrem após não receber nenhuma propo...              0     0
2    @netflixbrasil se vc tirar naruto vc me paga n...              1     1
3     faltou o loirinho pra assistir netflix cmg askbf              0     0
4    netflix se as personagens são portugueses não ...              1     0
..                                                 ...            ...   ...
189  @deltanz eu lembro que tinha isso isso eh comu...              0     0
190  alguém on pra indicar uma coisa mto boa pra ve...              0     0
191                          pelo amor de deus netflix              1     0
192  @soy_qiima vai se fuder todo filme da netflixn...              1     0
193      hoje só um japa e netflix mudaria o meu humor              1     1

[194 rows x 3 columns]

**A tabela abaixo apresenta as porcentagens dos resultados do teste nas seguintes categorias:**

- Verdadeiros negativos: **Linha 1 ; Coluna 1**
- Falsos positivos: **Linha 1 ; Coluna 2**
- Falsos negativos: **Linha 2 ; Coluna 1**
- Verdadeiros positivos: **Linha 1 ; Coluna 1**

In [19]:
pd.crosstab(test_final['classificação'],test_final['teste'],normalize = True).round(4)*100

teste              0      1
classificação              
0              34.02  18.56
1              17.53  29.90

**Para este teste obtivemos as seguintes porcentagens por tipo de resultado:**

Verdadeiros negativos: **34,02%**

Falsos positivos: **18,56%**

Falsos negativos: **17,53%**

Verdadeiros positivos: **29,9%**

**Resultando em uma taxa de acerto de: 63,9%** 

___
### Concluindo

- **Como observado nos percentuais acima, o classificador conseguiu atingir uma performance razoável se comparado a um classificador aleatório, que tenderia a obter um precisão de 50%. Quando analiza-se os percentuais de resultados verdadeiros, pode-se verificar que apresentam resultados próximos, o que nos indica que quando o classificador obteve sucesso, a distribuição de resultados é próxima da real(aproximadamente metade dos tweets era relevante e metade era irrelevante). Por outro lado, ao observar os percentuais de resultados falsos, o classificador apresenta percentuais próximos para falsos positivos e falsos negativos, esse fato indica que o classificador, ao errar, não teve preferência por um tipo de erro específico, ou seja, não aparenta ter um viés para os erros.**
___

- **Tratamento de dupla negação e sarcasmo:** O classificador Naive-Bayes é considerado "ingênuo" pois ele trata as palavras de um texto de forma individual, sem considerar o contexto em que estão inseridas, levando em conta apenas seu significado isolada das demais que compõe tal texto, nesse caso um tweet. Portanto este classificador na sua forma mais simples é incapaz de interpretar o contexto em que uma palavra está inserida para inferir seu real significado. Alem disso tal incapacidade tambem o limita de identificar que duplas negações podem significar uma afirmação pois estas negações são tratadas independentemente uma da outra.
___

- **Plano de expansão:** Em uma primeira iteração, nosso classificador já conseguiu atingir uma precisão razoável mesmo possuindo métodos de limpeza e análise considerados simples, isto é um indicativo do potencial do classificador. Do ponto de vista da equipe de marketing de uma grande empresa como a netflix, é de extrema importância saber o sentimento geral dos consumidores acerca de seus produtos, nosso classificador se apresenta como um meio de automatizar o processo de filtragem de informações que serão úteis à empresa para a tomada de decisões estratégicas com a finalidade de atender às necessidades de sua base de clientes. Portanto, um investimento continuado no projeto pode trazer resultados ainda mais precisos de forma a garantir uma maior qualidade de informações à empresa.
___

- **Outros cenários de Naive-Bayes:** Alem de um classificador de tweets com base em relevância, outro cenário em que um classificador Naive-Bayes poderia ser usado é na criação de um filtro de spam para e-mails. O classificador consegue aprender, a partir de uma base de dados, quais são as palavras mais recorrentes em e-mails considerados spam e consequentemente consegue classificar novos e-mails como spam(são descartados) ou não-spam(direcionados à caixa de entrada). Outro cenário em que um classificador Naive-Bayes pode ser usado é na classificação de artigos científicos por área do conhecimento a partir das palavras-chave, identificando quais são as palavras-chave mais recorrentes em artigos de cada área e posteriormente classificando novos artigos.
___

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**